# Introduction
* Reinforcement learning works by **trial and error** - aiming to maxmize a reward function/high score
* It is **unsupervised**, and wanders around the problem space establishing structure to maximize its reward

### Definitions
* **Environment** - The problem space, like a video game or financial market
* **State** - All relevant parameters defining the problem space
* **Agent** - **All Elements** of the algorithm that interact with the state
* **Action** - Choose action ${A_{i}}$ from the set of the actions
* **Step** - Given the Agent's *action*, the *environment* is updated - this is one *step* 